# Modul Daten

Dieses Jupyter Notebook soll einen Überblick über die Aufbereitung, Bereinigung und Visualisierung von Daten vermitteln. 

Für diesen Zweck wurde die Datenbank Gas Sensor Array Drift verwendet, heruntergeladen und vorher für diesen Zweck manipuliert (Gas- und Konzentrationsklassenlabel wurden zu einem Label zusammen geführt und dabei drei verschiedene Konzentrationsstufen definiert: low, medium und high). <br>
Der vollständige Datensatz kann auf <br>

http://archive.ics.uci.edu/ml/datasets/Gas+Sensor+Array+Drift+Dataset+at+Different+Concentrations

heruntergeladen werden. Außerdem werden verschiedene Informationen zu den Daten bereitgestellt. <br>

Der Datensatz enthält 13.910 Messungen von 16 chemischen Sensoren, welche abwechselnd sechs verschiedene Gase (Ethanol, Ethen, Ammoniak, Acetaldehyd, Aceton und Toluol) in verschiedenen Konzentrationen untersuchen. <br>
Der Datensatz wurde zwischen Januar 2008 und Februar 2011 (36 Monaten) erhoben. <br>
Jede Messung liefert anhand der 16 vorhandenen Sensoren eine 16-Kanal-Zeitreihe. <br>
Es werden zwei Hauptfeatures in diesem Datensatz betrachtet: <br>
$\rightarrow~$ (i) das stationäre Feature (steady-state) bezeichnet als `DR`, definiert als die maximale Widerstandsänderung in Bezug auf eine Basis, sowie die normalisierte Version davon (`|DR|`). <br>
$\rightarrow~$ (ii) ein Ansammlung an Features welche die Sensordynamik der gesamten Messung wiederspiegelt (`EMAi` und `EMAd` für verschiedene $\alpha$-Werte). <br>

Im folgendem Beispiel werden wir uns nur mit dem `DR` Feature auseinandersetzen. Da jeder der 16 Sensoren diesen Messwert liefert, sind auch 16 verschiedene Messwerte von `DR` vorhanden "(`DR_1` des ersten Sensors, `DR_2`, des zweiten Sensors und so weiter). 

## Daten laden

Als erstes müssen alle Beispieldatensätze geladen werden. Dieser liegen in 10 verschiedene csv Dateien vor (batch0.csv bis batch9.csv). Anschließend werden diese zu einem gemeinsamen Datensatz zusammengeführt.

In [ ]:
!git clone https://github.com/r1marcus/TraintheTrainer-Daten.git
!echo $CWD
!cp -rf /content/TraintheTrainer-Daten/* /content/

import scripts.load
df_modified, df_original,all_files_mod,all_files_original = scripts.load.load()
import scripts.splitt
df_training, df_test = scripts.splitt.splitt(all_files_mod,70,30)

## Daten Bereinigen und Transformieren

In diesem Abschnitt soll eine sehr elegante Methode zur Bereinigung und Transformation der Daten vorgestellt werden. <br>
Da hier eine Sequenz an verschiedenen Transformationen durchgeführt werden soll, bietet sich die Anwendung von SciKit-Learn's Pipeline an. <br>


#### NaNs
Für den Umgang mit NaNs gibt es einige Möglichkeiten: <br>
$\rightarrow~$ Entfernen der jeweiligen Instanzen (Pandas `dropna()`). <br>
$\rightarrow~$ Entfernen des ganzen Features (Pandas `drop()`). <br>
$\rightarrow~$ Ersetzen durch einen bestimmten Wert (Null, den Median, den Mittelwert mittels Pandas `fillna()` oder Scikit-Learns SimpleImputer Transformer)<br>
Im folgenden Beispiel werden wir NaNs durch den Median ersetzen. <br>

#### Ausreißer
Um Ausreißer zu detektieren kann die $\sigma$-Regel verwendet werden. Dabei wird der sogenannte Z-Score für jeden Datenpunkt berechnet (die Entfernung eines Datenpunktes zum Mittelwert ausgedrückt in Standardabweichungen) und falls der Betrag des Z-Scores einen Grenzwert überschreitet (z.B. drei, heißt drei Standardabweichungen von Mittelwert entfernt) kann dieser Datenpunkt als Ausreißer betrachtet werden. <br>
Eine andere Möglichkeit wäre einfach ein gültigen Wertebereich von Hand zu definieren: [$x_{\text{min}},x_{\text{max}}$] und alle Werte außerhalb des Gültigkeitsbereichs als Aureißer zu betrachten. <br>
Sind Ausreißer erkannt worden, können diese genau wie NaNs behandelt werden. Ausreißer werden in diesem Beispiel durch den Median ersetzt. <br>


#### Rauschen
Anschließend soll kurz auf die Behandlung von Rauschen eingegangen werden. Rauschen kann bereits Hardwareseitig durch Frequenzfilter (z.B. diskrete Lineare Filter wie den Butterworth-Filter oder den exponentiellen Filter) entfernt werden. Aber auch Softwareseitig werden verschiedene Filtermethoden angeboten. In der Regel werden bei der Anwendung solcher Filter auf Zeitreihen alle Daten verändert. Ein Beispiel ist der gleitenden Mittelwert oder der gleitenden Median. Letzteres soll hier vorgestellt werden, da Pandas diesen bereits mitbringt. <br>

An dieser Stelle möchte ich erwähnen das die Behandlung von Rauschen hier nur zu demonstrationszwecken gezeigt werden soll. In einem Realfall wäre der Einsatz für diesen Datensatz mehr als fraglich, da der gleitende Mittelwert hier über verschiede Batches, verschiedene Gase und verschiedene Konzentrationsstufen hinaus berechnet wird, was offensichtlich nicht sinnvoll ist. <br>


#### Normalisierung und Skalierung
Im letzten Schritt der Datenbereinigung werden wir die Daten skalieren. Verschiedene Features im Datensatz können Werte in verschiedenen Bereichen aufweisen. So kann beispielsweise in einem Mitarbeiterdatensatz der Gehaltsbereich von Tausend bis Hunderttausend liegen, der Wertebereich des Altersmerkmals aber nur zwischen 20-70. Das heißt, ein Feature ist im Vergleich zum anderen stärker gewichtet. Anwendungen statistischer Methoden können in solchen Situationen unerwünschte und unbrauchbare Ergebnisse liefern. Anwendungen bei denen eine Featureskalierung wichtig ist sind <br>
$\rightarrow~$ k-nearest neighbors oder k-means mit euklidischem Abstandsmaß; <br>
$\rightarrow~$ Logistische Regression, SVM, Perzeptrons oder neuronale Netze bei denen Gradient Descent verwendet wird; <br>
$\rightarrow~$ lineare Diskriminanzanalyse, Hauptkomponentenanalyse oder Kernel-Hauptkomponentenanalyse. <br>

Um das zu vermeiden empfiehlt es sich die Daten zu transformieren. Gängige Transformationen sind <br>

$\rightarrow~$ Standardisierung: skalieren der Daten auf die Normalverteilung $\mathcal{N}(0,1)$; <br>
$\rightarrow~$ Min/Max-Normalisierung: skalieren der Daten auf ein fixen Bereich, z.B. [0,1]; <br> 
$\rightarrow~$ $\mathcal{l}_{1}$-/$\mathcal{l}_{2}$-Normalisierung: Normalisierung des Featurevektors auf den Einheitsvektor (bei der $\mathcal{l}_{1}$-Normalisierung summiert sich der Absolutwert jedes Elements auf 1 und bei der $\mathcal{l}_{2}$-Normalisierung summiert sich die Quadratsumme zu 1).

Welche letztlich die beste Wahl ist hängt stets von der Situation ab.

### Pipelines

Das hier gezeigte Vorgehen versteift sich auf die Anwendung von Pipelines (siehe Backup der Modul Daten Präsentation). <br>
Diese können eine Sequenz von Transformationen (sogenannte Transformatoren) abarbeiten. Zudem können schnell und einfach eigene Transformatoren geschrieben werden, wie hier gezeigt für den `RollingMean`, die `OutlierDetection` und den `GasLabel` Transformer. <br>

Das implementieren eines eigenen Transformers soll am Beispiel `RollingMean` kurz beschrieben werden. Als erstes wird eine Klasse mit dem Namen RollingMean definiert, welche von den Basisklassen BaseEstimator und TransformerMixin erbt. Die `fit()` Methode gibt einfach self zurück und lediglich die `transform()` Methode wird der Anforderung nach angepasst. Hier wird ein DataFrame Objekt zurückgegeben welches den asymmetrischen gleitenden Mittelwert enthält.

```python
class RollingMean(BaseEstimator, TransformerMixin):
    def __init__(self, window_size=3):
        self._window = window_size
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        df_temp = pd.DataFrame(X)
        return df_temp.rolling(window=3, min_periods=1).mean().to_numpy()

class OutlierDetection(BaseEstimator, TransformerMixin):
    def __init__(self, std):
        self._std = std
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        mask = np.abs((X - X.mean(0)) / X.std(0)) > 3
        return np.where(mask, np.median(X, axis=0), X)
    
 ```   

Als erstes sollte berücksichtigt werden, dass numerische Features anders als (textbasierte) kategorische Features oder Klassenlabels tranformieren. Wir trennen daher die Klassenlabels von den restlichen Daten, Prädiktoren genannt. Die Prädiktoren können je nach Typ, numerisch oder textbasiert, weiter getrennt werden. In unserem Beispiel sind die Prädiktoren numerisch. Wir trennen also lediglich die Klassenlabels von den Prädikatoren und transformieren beide getrennt voneinander durch unterschiedliche Transformatoren und Pipelines. <br>

Wir definieren uns eine Liste mit Prädiktoren die transformiert werden sollen. Alle anderen werden nicht weiter berücksichtigt. Wir bezeichnen diese Liste als num_features und tragen dabei die Namen der Features ein, hier die `DR` Messungen der ersten vier Sensoren. Da wir auch die Klassenlabels transofmieren möchten erstellen wir für diese ebenfallls eine Liste.

In [ ]:
num_features = ['DR_1', 'DR_2', 'DR_3', 'DR_4']

#### Transformationen der numerischen Features

Anschließend definieren wir die Pipeline der numerischen Transformationen als Liste. Diese erhält den Namen des Transformers (beliebig) sowie den Transformer. Dies legt auch die Reihenfolge der durchzuführenden transformationen fest.
```python
num_pipeline = Pipeline([
		('mean_imputer', SimpleImputer(strategy='median')),
		('outlier_detection', OutlierDetection(std=3)),
		('rolling_mean', RollingMean(3)),
		('scaler', MinMaxScaler(feature_range=(0,1))),
	])
```

$\rightarrow~$ SimpleImputer: ersetzt NaNs durch den Median. <br>
$\rightarrow~$ OutlierDetection: detektiert Ausreißer nach der 3-$\sigma$ Regel und ersetzt diese durch den jeweiligen Median des Features. Die Implementierung ist oben gezeigt. <br>
$\rightarrow~$ RollingMean: behandelt Daten mit rauschen mit dem gleitenden Median und einer Fenstergröße von drei Werten. <br>
$\rightarrow~$ MinMaxScaler: Skaliert uns die Daten auf ein gewünschtes Intervall, hier auf das Intervall [0,1].

#### Transformationen der Klassenlabels

Erstellen wir noch eine Pipeline, welche die transformationen der Klassenlabels festlegt.
```python
target_pipeline = Pipeline([
		('imputer', SimpleImputer(strategy='most_frequent')),
		('one_hot_encoder', OneHotEncoder(sparse=False, categories='auto')),
		('sorter', GasLabel()),
	])
    
```    

$\rightarrow~$ SimpleImputer: ersetzen fehlender Werte durch den meist auftretenden Wert (äquivalent zur Mode, andere statistische Maße die genutzt werden können sind Mean oder Median). Der SimpleImputer soll hier nur zu Präsentationszwecken gezeigt werden, da keine Klassenlabels im vorliegenden Beispieldatensatz fehlen. <br>
$\rightarrow~$ OneHotEncoder: überführt textbasierte kategorische Daten zunächst in numerische Kategorien und One-Hot encoded diese anschließend. Das heißt, es werden $n$ binäre Spaltenvektoren ($n$ Anzahl der verschiedenen Klassenlabels, hier 18) erzeugt. Das One-Hot encoding macht Sinn, da die meisten Algorithmen des Machine-Learnings den Umgang mit Zahlen bevorzugen. <br>
$\rightarrow~$ Zuletzt habe ich den `GasLabel` Transformer implementiert. Dieser fügt am Ende eine weitere Spalte hinzu und ist vom Prinzip her das selbe wie Scikit-Learns `LabelEncoder` (dieser funktioniert im aktuellen Release nicht mit dem ColumnTransformer den wir später verwenden werden). Dabei wird jeder Kategorie eine Zahl zugeordned, aufsteigend und beginnend mit eins. Dies wird uns später bei der Datenvisualisierung helfen.

#### ColumnTransformer

Sind beide Pipelines definiert, führen wir diese mit dem `ColumnTransfromer` zusammen und geben für jede Pipelinde noch die zu transformierende Liste mit. <br>
```python
full_pipeline = ColumnTransformer(
		transformers=[
			('num', num_pipeline, num_features),
			('target', target_pipeline, target)],
		remainder='drop')
```


Transformieren wir die Trainungs- und Testdaten. Beachte, wir erhalten kein Pandas DataFrame Objekt als Rückgabewert sondern NumPy Arrays.

In [ ]:
import scripts.clean
import numpy as np

np_train_prepared,np_test_prepared = scripts.clean.clean(df_training,df_test,num_features)

Überprüfen wir ob die Dimensionen stimmen und ob noch NaNs vorhanden sind.

In [ ]:
np_train_prepared.shape, np_test_prepared.shape

In [ ]:
np.isnan(np.min(np_train_prepared)), np.isnan(np.min(np_test_prepared))

Untersuchen wir die ersten zwei Zeilen der transformierten Trainingsdaten.

In [ ]:
np_train_prepared[:2]

Sollte eine fortgeschrittene Feature Selection Teil der Pipeline sein, kann auf https://scikit-learn.org/stable/modules/feature_selection.html mehr dazu gelesen werden.

In [ ]:
df_training.shape, df_original.shape

## Speichern der bereinigten Daten

Speichern wir Daten im csv und pickle Format ab:

In [ ]:
scripts.clean.safe(np_train_prepared,np_test_prepared)

Nach dem bereinigen, transformieren und speichern der Daten, widmen wir uns als nächstes verschiedene Möglichkeiten der Datenvisualisierung.

## Do it yourself:


Ändern Sie die Features und überprüfen Sie ob die Pipeline immer noch funktioniert. Was sind die Vorteile einer einer Pipeline?
Nutzen Sie statt den Sensoren 1-4 die Sensoren 8-12